In [1]:
import phidl.geometry as pg
from phidl.quickplotter import quickplot2 as qp2
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon,Rectangle
from phidl.path import Path,CrossSection
import numpy as np
import shapely
import shapely as sh
from centerline.geometry import Centerline
from scipy.spatial import cKDTree
import warnings
import networkx as nx
import utilities as utils
%matplotlib qt
palette={'black': '#000000', 'vermillion': '#E69F00',
'blue': '#56B4E9', 'green': '#009E73','orange': '#D55E00',
'yellow': '#F0E442', 'red': '#CC79A7','darkgrey':'#2F4F4F'}

poly_color=palette['blue']

In [2]:
def smooth_boolean(dev,radius):
    return utils.shapely_to_phidl(utils.phidl_to_shapely(dev).buffer(radius).buffer(-radius))


In [3]:
n_ondulation = [16, 16, 8]
rs = [550, 320, 120]
r_mod = [0.06, 0.06, 0.06]
thicknesses = [25,15,10]

dev=pg.Device()
polys=[]
connection_areas=pg.Device()
connection_area_rad=35

for idx, (rad, thick) in enumerate(zip(rs, thicknesses)):
    r = rad + r_mod[idx] * rad * np.sin(np.linspace(0, n_ondulation[idx] * 2 * np.pi, 512))
    initial_rot = 30
    t = np.linspace(0, 2 * np.pi, 512) + initial_rot
    x, y = r * np.cos(t), r * np.sin(t)
    plt.plot(x,y)
    pth = Path(np.vstack([x,y]).T)
    X=CrossSection()
    X.add(width=thick)
    buffer_out=utils.phidl_to_shapely((pth.extrude(X))).buffer(distance=20).buffer(-20)
    dev<<utils.shapely_to_phidl(buffer_out)
    polys.append(np.vstack((x,y)).T)

paths=pg.Device()
for idx,n in enumerate([16,8]):
    for latch in range(n):
        x0,y0 = polys[idx][len(polys[idx])//n*latch]
        xe,ye = polys[idx+1][(len(polys[idx+1])//n*latch+65)%len(polys[idx+1])]
        
        connection_areas<<pg.circle(radius=connection_area_rad).movex(xe).movey(ye)
        connection_areas<<pg.circle(radius=connection_area_rad).movex(x0).movey(y0)

        
        r0,re = np.sqrt(x0*x0+y0*y0),np.sqrt(xe*xe+ye*ye)
        t0,te = np.arctan2(y0,x0),np.arctan2(ye,xe)
        if (te-t0)<0: #looping the other way around
            t0-=2*np.pi
        t=np.linspace(t0,te,100)
        rz=np.linspace(0,1,100)*(re-r0)+r0
        x_latch,y_latch = rz*np.cos(t),rz*np.sin(t)
        plt.plot(x_latch,y_latch)
        X=CrossSection()
        X.add(width=10)
        paths<<Path(np.vstack((x_latch,y_latch)).T).extrude(X)
joined_dev = pg.boolean(dev,paths,operation='A+B')
        
smooth_connections = pg.boolean(joined_dev,connection_areas,operation='AND') #cut out contact areas
smooth_connections=utils.phidl_to_shapely(smooth_connections).buffer(20).buffer(-20)

utils.shapely_to_phidl(utils.phidl_to_shapely(joined_dev).union(smooth_connections).buffer(5).buffer(-5))

final_dev = smooth_boolean( pg.boolean(smooth_connections,joined_dev,operation='A+B'), radius=5)

tail = pg.rectangle( (400,80) ).movex(-200).movey(900)
tail = smooth_boolean( pg.boolean( tail, pg.rectangle( (70,400)).movex(-35).movey(530),operation='A+B'),radius=5)

device_perimeter=pg.Device()
device_perimeter.add_polygon(polys[0])

tail = pg.boolean(tail,device_perimeter,operation='A-B')

final_dev = smooth_boolean( pg.boolean(final_dev,tail,operation='A+B'), radius=5)
qp2(final_dev)

pads=pg.Device()
pads.add_array(pg.rectangle((10,50)).movex(-195).movey(930),columns=19,rows=1,spacing=(21,0))

tmp=pg.Device()
tmp<<final_dev
tmp<<pads
qp2(tmp)

/home/csaba/anaconda3/lib/python3.12/site-packages/phidl/quickplotter.py:959: UserWarning: [PHIDL] Warning: `quickplot2` will be removed in April 2025, please replace with quickplot() 
  warnings.warn(


In [4]:
shapely_device=utils.phidl_to_shapely(final_dev)

In [5]:
router= utils.RoutingLayout(shapely_device,border_tolerance=1,skel_interp_dist=1,override_auto_skel_dist=True)


In [6]:
router.lay_down_lines(9,3,add_border=False)


In [7]:
qp2(utils.multilinestrings_to_phidl(router.all_route_lines))

In [8]:
electrode_xy=np.vstack((polys[-1][::512//3,:][:-1,:],polys[-2][(55+512//6*np.arange(6))%512,:])).T
elec_boundaries=[]
elec_extra_points=[]
for x,y in electrode_xy.T:
    elec_boundaries.append(shapely.Point(x,y).buffer(5).boundary)
    elec_extra_points.append(shapely.MultiPoint([shapely.Point(x,y)]))
router.add_electrodes(elec_boundaries,elec_extra_points,clearance=2,type='electrode')


In [9]:
pad_points=shapely.LineString([shapely.Point([-10,650]),shapely.Point([10,650])]).intersection(router.all_route_lines)
pad_boundaries=[]
pad_extra_points=[]
for p in pad_points.geoms:
    x,y=list(p.coords)[0]
    pad_boundaries.append( shapely.Point(x,y).buffer(1).boundary)
    pad_extra_points.append(shapely.MultiPoint([shapely.Point(x,y)]))
router.add_electrodes(pad_boundaries,pad_extra_points,clearance=1,type='pad')

In [10]:
router.route()

In [11]:
for idx,line in enumerate(router.all_route_lines.geoms):
    x,y=np.asarray(line.coords).T
    if np.min( (x+196.9)**2+(y+117.2)**2) < 1e-1: #select specific line
        line_idx=idx

latch=router.all_route_lines.geoms[line_idx].buffer(30).intersection(shapely_device).difference(utils.phidl_to_shapely(dev))
latch=latch.geoms[3]
midline=router.all_route_lines.geoms[line_idx].segmentize(1)
x,y=np.asarray(midline.coords).T
norm=np.asarray([np.gradient(y),-np.gradient(x)])
norm=norm/(np.sqrt(np.sum(norm**2,axis=0)))
s=np.cumsum(np.sqrt(np.gradient(x)**2+np.gradient(y)**2))
s=s/np.max(s)
scaler=np.tanh((s-0.1)*15)+np.tanh((-s+0.9)*15)
scaler-=np.min(scaler)
scaler/=np.max(scaler)
mag=3*scaler
fig,ax=plt.subplots(1,1,figsize=(5,5))
ax.set_aspect('equal')
newx,newy=x+mag*np.sin(15*np.pi*2*s)*norm[0],y+mag*np.sin(15*np.pi*2*s)*norm[1]
ax.plot(newx,newy)
np.sum( norm**2, axis = 0 )

wiggled_line=shapely.LineString(np.array([newx,newy]).T)

line_idx_wig=line_idx

In [12]:
from matplotlib.gridspec import GridSpec
fig = plt.figure(layout="constrained")

gs = GridSpec(4, 3, figure=fig)
ax1 = fig.add_subplot(gs[:-1, :2])
ax1.set_axis_off()

ax2=fig.add_subplot(gs[:-1,-1])
ax3=fig.add_subplot(gs[-1,:])
for a in [ax1,ax2,ax3]:
#     a.set_aspect('equal')
     a.set_axis_off()
import matplotlib.patches as mp
ax1.set_xlim(-600,600)
ax1.set_ylim(-600,960)
ax2.set_xlim(-340,-260)
ax2.set_ylim(-170,10)
ax3.set_xlim(-340,-340+340)
ax3.set_ylim(-136,-50)
ax2_box_shapely = shapely.Polygon(np.asarray([[(i,j) for i in ax2.get_xlim()] for j in ax2.get_ylim()]).reshape(4,2)[[0,1,3,2],:])
ax3_box_shapely = shapely.Polygon(np.asarray([[(i,j) for i in ax3.get_xlim()] for j in ax3.get_ylim()]).reshape(4,2)[[0,1,3,2],:])

for p in final_dev.get_polygons():
    ax1.add_patch(mp.Polygon(p,fc=poly_color))
#     ax2.add_patch(mp.Polygon(p,fc=poly_color))
#     ax3.add_patch(mp.Polygon(p,fc=poly_color))
ax2_dev=shapely_device.intersection(ax2_box_shapely)
ax3_dev=shapely_device.intersection(ax3_box_shapely)
ax2.add_patch(mp.Polygon( np.asarray(ax2_dev.exterior.coords),fc=poly_color))
ax3.add_patch(mp.Polygon( np.asarray(ax3_dev.exterior.coords),fc=poly_color))

for k,v in router.G.edges:
    try:
        line_idx=np.where(np.sum(router.end_to_end_connections==k,axis=1) + np.sum(router.end_to_end_connections==v,axis=1) == 2)[0][0]
        simplified_line = router.all_lines.geoms[line_idx].simplify(.1) #save memory for .svg
        
        xs,ys=np.asarray(simplified_line.simplify(1).coords).T
        x,y=np.asarray(simplified_line.coords).T
        if line_idx!=line_idx_wig:
            ax1.plot(xs,ys,color='k',linewidth=.3)
        if simplified_line.intersects(ax2_box_shapely):
            ax2.plot(x,y,color='k',linewidth=1)
        if simplified_line.intersects(ax3_box_shapely):
            if line_idx==line_idx_wig:
                ax1.plot(newx,newy,color='r',linewidth=.5)
                ax3.plot(newx,newy,color='r',linewidth=1)
            else:
                ax3.plot(x,y,color='k',linewidth=1)
    except:
        continue

elec_xy=[-319.3,-98.5]
t=np.linspace(0,2*np.pi,100)

cx,cy = np.cos(t)*5+elec_xy[0],np.sin(t)*5+elec_xy[1]
cx2,cy2 = np.cos(t)*7+elec_xy[0],np.sin(t)*7+elec_xy[1]

ax2.plot(cx,cy,color='goldenrod')
ax2.plot(cx2,cy2,color='r',linestyle='--')

plt.savefig('Fig2a.svg')



In [13]:
router= utils.RoutingLayout(shapely_device,border_tolerance=1,skel_interp_dist=1,override_auto_skel_dist=True)
router.lay_down_lines(19,1.5,add_border=False)


electrode_xy=np.vstack((polys[-1][::512//6,:][:-1,:],polys[-2][(55+512//13*np.arange(13))%512,:])).T
pad_points=shapely.LineString([shapely.Point([-20,650]),shapely.Point([20,650])]).intersection(router.all_route_lines)

elec_boundaries=[]
elec_extra_points=[]
for x,y in electrode_xy.T:
    elec_boundaries.append(shapely.Point(x,y).buffer(5).boundary)
    elec_extra_points.append(shapely.MultiPoint([shapely.Point(x,y)]))
router.add_electrodes(elec_boundaries,elec_extra_points,clearance=2,type='electrode')

pad_points=shapely.LineString([shapely.Point([-20,650]),shapely.Point([20,650])]).intersection(router.all_route_lines)
pad_boundaries=[]
pad_extra_points=[]
for p in pad_points.geoms:
    x,y=list(p.coords)[0]
    pad_boundaries.append( shapely.Point(x,y).buffer(1).boundary)
    pad_extra_points.append(shapely.MultiPoint([shapely.Point(x,y)]))
router.add_electrodes(pad_boundaries,pad_extra_points,clearance=1,type='pad')

In [14]:
router.route()

Routing successful


In [15]:
fig = plt.figure(layout="constrained")

from matplotlib.gridspec import GridSpec
fig = plt.figure(layout="constrained")

gs = GridSpec(3, 3, figure=fig)
ax1 = fig.add_subplot(gs[:, :2])
ax2=fig.add_subplot(gs[:,-1])

ax1.set_axis_off()
ax2.set_axis_off()

ax1.set_xlim(-600,600)
ax1.set_ylim(-600,960)

_x0,_x1,_y0,_y1 = -340,-260,-170,10
ax2.set_xlim(-340,-260)
ax2.set_ylim(-170,10)

for p in final_dev.get_polygons():
    ax1.add_patch(mp.Polygon(p,fc=poly_color))
    ax2.add_patch(mp.Polygon(p,fc=poly_color))

#add effective grid of possible routing:


ax2_fov = sh.Polygon([[_x0,_y0],[_x1,_y0],[_x1,_y1],[_x0,_y1]])
for k,v in router.G.edges:
    try:
        line_idx=np.where(np.sum(router.end_to_end_connections==k,axis=1) + np.sum(router.end_to_end_connections==v,axis=1) == 2)[0][0]
        linest=router.all_lines.geoms[line_idx]
        if linest.intersects(ax2_fov): #reduce time to plot & fig memory
            ax2.plot(*linest.xy,linestyle='dashed',color='k',linewidth=.5)
    except:
        continue



#top part of routing
sorted_leftright=np.argsort([float(el.xy[0][0]) for el in pad_points.geoms])
staircase=np.cumsum( np.ones(len(electrode_xy[0]))*(-np.sign(2*(np.arange(len(electrode_xy[0]))//10) -1))) #1,2,3,4,...,4,3,2,1
for n,idx in enumerate(sorted_leftright):
    pt=pad_points.geoms[idx]
    linestring = sh.LineString([pt,
                                sh.Point(pt.x,pt.y+250+3*staircase[n]),
                                sh.Point(pt.x-171+19*n,pt.y+250+3*staircase[n]),
                                sh.Point(pt.x-171+19*n,970)])
    ax1.add_patch(Rectangle(xy=[pt.x-171+19*n-6,940],width=12,height=32,color='gold'))
    ax1.plot(*linestring.xy,color='gold',linewidth=.5)
    ax1.plot(*router.all_elec_pad_routes[n].xy,color='gold',linewidth=.5)
    ax2.plot(*router.all_elec_pad_routes[n].xy,color='gold')
    ax2.plot(electrode_xy[0][n],electrode_xy[1][n],'o',markersize=15,color='gold')
ax1.plot(*shapely_device.exterior.xy)
plt.savefig('fig2b.svg',dpi=300)